In [11]:
# Import libraries
import py_stringsimjoin as ssj
import py_stringmatching as sm
import pandas as pd
import os, sys

In [12]:
recordLinkagePath = '/media/ravirajukrishna/Windows/Users/Ravi/Desktop/USC/Courses_Sem3/Info_Integrate_Web/Project/Enroute-Genie/data_record_linkage_csv/california/'
blogTablePath = recordLinkagePath + 'allBlogLocation_nonCity_Mentions.csv'
tripAdvisorTablePath = recordLinkagePath + 'tripAdvisor_attractions.csv'
# Load csv files as dataframes.
blogTable = pd.read_csv(blogTablePath)
tripAdvisorTable = pd.read_csv(tripAdvisorTablePath)
print('Number of records in blogTable: ' + str(len(blogTable)))
print('Number of records in tripAdvisorTable: ' + str(len(tripAdvisorTable)))

Number of records in blogTable: 7395
Number of records in tripAdvisorTable: 5445


In [13]:
# profile attributes in table blogTable
ssj.profile_table_for_join(blogTable)

,Unique values,Missing values,Comments
Attribute,,,
mention_name,7395 (100.0%),0 (0.0%),This attribute can be used as a key attribute.
mention_type,263 (3.56%),0 (0.0%),
mention_popularity,97 (1.31%),0 (0.0%),
mention_address,6341 (85.75%),0 (0.0%),
no_of_blogFiles,181 (2.45%),0 (0.0%),
mention_found_in_blogFiles,3503 (47.37%),0 (0.0%),


In [14]:
# profile attributes in table tripAdvisorTable
ssj.profile_table_for_join(tripAdvisorTable)

,Unique values,Missing values,Comments
Attribute,,,
attraction_name,5388 (98.95%),0 (0.0%),
attraction_knownFor,751 (13.79%),3856 (70.82%),Joining on this attribute will ignore 3856 (70.82%) rows.
attraction_reviewComment,5045 (92.65%),374 (6.87%),Joining on this attribute will ignore 374 (6.87%) rows.
attraction_ranking,361 (6.63%),0 (0.0%),
attraction_no_of_reviews,579 (10.63%),306 (5.62%),Joining on this attribute will ignore 306 (5.62%) rows.
attraction_contact,3973 (72.97%),1354 (24.87%),Joining on this attribute will ignore 1354 (24.87%) rows.
attraction_in,682 (12.53%),0 (0.0%),
attraction_address,4859 (89.24%),329 (6.04%),Joining on this attribute will ignore 329 (6.04%) rows.
attraction_url,5445 (100.0%),0 (0.0%),This attribute can be used as a key attribute.


In [15]:
# create whitespace tokenizer for tokenizing 'address' attribute. The return_set flag should be set to True since
# Jaccard is a set based measure.
ws = sm.WhitespaceTokenizer(return_set=True)

# a whitespace tokenizer will tokenize the input string using whitespace
#ws.tokenize('Lake Ilsanjo, Lake Trail, Sonoma County, California, 95409, United States of America')

In [21]:
# find all pairs from blogTable and tripAdvisorTable such that the Jaccard score
# on 'name' is at least 0.5.
# l_out_attrs and r_out_attrs denote the attributes from the 
# left table (A) and right table (B) that need to be included in the output.

output_pairs = ssj.jaccard_join(blogTable, tripAdvisorTable, 
                                'mention_name', 'attraction_url', 
                                'mention_name', 'attraction_name', 
                                ws, 0.38, 
                                l_out_attrs=['mention_name','mention_type','mention_popularity'], 
                                r_out_attrs=['attraction_name','attraction_knownFor','attraction_reviewComment'],
                                n_jobs=-1)
print(len(output_pairs))
# examine the output pairs
(output_pairs)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


4


,_id,l_mention_name,r_attraction_url,l_mention_type,l_mention_popularity,r_attraction_name,r_attraction_knownFor,r_attraction_reviewComment,_sim_score
0,0,beach,https://www.tripadvisor.com/Attraction_Review-g152515-d10723455-Reviews-Divorce_beach-Cabo_San_L...,service,4.0,Divorce beach,"“ Goes hand in hand with Lovers beach ” Reviewed September 17, 2016 via mobile If you've been to...",NaN,0.5
1,1,downtown historic district,https://www.tripadvisor.com/Attraction_Review-g32595-d4090017-Reviews-Historic_downtown_Lake_Els...,proposed,1.0,historic downtown Lake Elsinore,NaN,LOVED THE ENTIRETY OF THE WHOLE CITY! Wonderful place! will go again soon hopefully! many differ...,0.4
0,2,davis,https://www.tripadvisor.com/Attraction_Review-g33096-d6847417-Reviews-Davis_ranch-Sloughhouse_Ca...,administrative,4.0,davis ranch,NaN,"Davis Ranch has t he best white sweet corn there is anywhere, period ! I wish they would stay op...",0.5
1,3,lake,https://www.tripadvisor.com/Attraction_Review-g32689-d9581575-Reviews-Ellis_lake-Marysville_Cali...,administrative,11.0,Ellis lake,NaN,As a teenager during the summers my grandfather lived in the area and we would visit the lake an...,0.5
